In [2]:
import pandas as pd
import datetime as datetime
import pprint
import os
import psycopg2
import requests
import boto3
import dotenv
from requests import get 
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlunparse
from bson.json_util import dumps
from IPython.display import display, Image
from IPython.display import HTML

# An example of getting current date

currDate = datetime.datetime.now()
 
print(currDate)

2023-06-13 21:39:42.478207


In [5]:
# Load the environment variables from the .env file
dotenv.load_dotenv()

scrape_url = os.getenv('SCRAPE_URL')
# print(scrape_url)

POSTGRES_URL = os.environ.get('POSTGRES_URL')
# print(POSTGRES_URL)


In [7]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from datetime import datetime

# Make a request to the webpage
# url = "https://www.example.com"  # Replace with your desired webpage URL
response = requests.get(scrape_url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text, "html.parser")

# Find all the lines (within <a> tags) in the webpage
lines = soup.find_all("a")

# Connect to the PostgreSQL database
# conn = psycopg2.connect(
#     host="your_host",
#     database="your_database",
#     user="your_user",
#     password="your_password"
# )

POSTGRES_URL = os.environ.get('POSTGRES_URL')
# print(POSTGRES_URL)

# Add 'sslmode=require' to the connection URL
POSTGRES_URL += "?sslmode=require"

# print(POSTGRES_URL)

# Insert keyboard interrupt to stop the execution
raise KeyboardInterrupt

conn = psycopg2.connect(POSTGRES_URL)

cursor = conn.cursor()

# Iterate over the lines and insert/update each line into the database
for line in lines:
    line_content = line.get_text()
    line_type = "Link"
    line_url = line.get("href")
    current_datetime = datetime.now()

    if line.find("img"):
        line_type = "Image"

    # Upsert the line into the database
    query = """
        INSERT INTO your_table (line_content, line_type, line_url, first_dt, latest_dt)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (line_content) DO UPDATE
        SET line_type = EXCLUDED.line_type,
            line_url = EXCLUDED.line_url,
            latest_dt = EXCLUDED.latest_dt
        WHERE your_table.latest_dt < EXCLUDED.latest_dt
    """
    data = (line_content, line_type, line_url, current_datetime, current_datetime)
    cursor.execute(query, data)

# Commit the changes and close the database connection
conn.commit()
cursor.close()
conn.close()


KeyboardInterrupt: 